In [3]:
SEASON = '1975-76'
LEAGUE = 'ABA'

In [4]:
import pandas as pd
import myconstants as c
from scipy.stats import zscore

NUM_GAMES = c.NUM_GAMES[SEASON]
MIN_GAMES = c.MIN_GAMES[SEASON]

p_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_{LEAGUE}_Player_PerGame.csv')
t_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_{LEAGUE}_Team_Standings.csv')
p_advanced_df = pd.read_csv(f'../data/years/{SEASON}/clean/{SEASON}_{LEAGUE}_Player_Advanced.csv')


# join player per game and advanced stats
p_advanced_df = p_advanced_df.drop(columns=['Player', 'Tms', 'Pos', 'Age', 'G', 'MP'])
p_df = p_df.merge(p_advanced_df, on='PlayerID')

# populate team df with team codes
t_df['TeamCode'] = t_df['Team'].apply(lambda x: c.TEAM_CODE[x])

# average team wins per player
def average_wins(tms):
    tms_list = tms.split(',')
    tms_wins = []
    for t in tms_list:
        tms_wins.append(t_df.loc[t_df['TeamCode'] == t]['OverallW'].values[0])
    return sum(tms_wins) / len(tms_wins)
p_df['TmsWins'] = p_df['Tms'].apply(average_wins)

# players who haven't played enough games don't qualify
p_df = p_df.loc[p_df['G'] >= MIN_GAMES]

# ranking algorithm
p_df['zTotals'] = 1.5*zscore(p_df['PTS'] + p_df['TRB'] + p_df['AST'] + p_df['STL'] + p_df['BLK'])
p_df['zWS'] = zscore(p_df['WS'])
p_df['zVORP'] = zscore(p_df['VORP'])
p_df['zBPM'] = zscore(p_df['BPM'])
p_df['zTmsWins'] = zscore(p_df['TmsWins'])
p_df['RawScore'] = p_df['zTotals'] + p_df['zWS'] + p_df['zVORP'] + p_df['zBPM'] + p_df['zTmsWins']
p_df['PlayerScore'] = p_df['RawScore'] + c.PLAYER_SCORE_ADDITION # more positive PlayerScores
p_df['PlayerScore'] = p_df['PlayerScore'].apply(lambda x: pow(x, c.PLAYER_SCORE_EXPONENT) if x > 1 else x) # make higher PlayerScores more significant
p_df = p_df.sort_values('PlayerScore', ascending=False)

# Move [PlayerScore] to the right of [Pos], drop PlayerID, index rows starting from 1, stylistic choice
col = p_df.pop('PlayerScore')
p_df.insert(p_df.columns.get_loc('Pos') + 1, col.name, col)
p_df = p_df.drop('PlayerID', axis=1)
p_df = p_df.reset_index(drop=True)
p_df.index = p_df.index + 1

p_df.head(50)

,Player,Tms,Pos,PlayerScore,Age,G,GS,MP,FG,FGA,...,DBPM,BPM,VORP,TmsWins,zTotals,zWS,zVORP,zBPM,zTmsWins,RawScore
1,Julius Erving,NYN,SF,112.129985,25,84,NaN,38.6,11.3,22.3,...,2.5,10.6,10.3,55.000000,4.015739,3.491050,4.702858,4.066326,0.977145,17.253117
2,Artis Gilmore,KEN,C,67.953483,26,84,NaN,39.1,9.2,16.7,...,0.8,4.7,5.5,46.000000,3.428709,2.805337,2.227669,1.876722,0.313873,10.652310
3,David Thompson,DEN,SF,58.216802,21,83,NaN,37.4,9.7,18.9,...,-0.2,3.7,4.4,60.000000,2.416081,2.093250,1.660439,1.505603,1.345629,9.021002
4,Bobby Jones,DEN,PF,55.427020,24,83,NaN,34.3,6.1,10.6,...,2.6,5.3,5.2,60.000000,1.506184,1.513032,2.072970,2.099393,1.345629,8.537208
5,Dan Issel,DEN,C,51.670609,27,84,NaN,34.0,9.0,17.5,...,-0.9,2.4,3.2,60.000000,2.342702,2.119624,1.041642,1.023147,1.345629,7.872744
6,James Silas,SAS,PG,50.933690,26,84,NaN,37.0,8.5,16.5,...,0.0,3.4,4.2,50.000000,1.902430,2.277866,1.557306,1.394267,0.608660,7.740528
7,Billy Paultz,SAS,C,45.018401,27,83,NaN,35.6,6.8,13.5,...,1.7,3.7,4.3,50.000000,1.814375,1.117428,1.608872,1.505603,0.608660,6.654939
8,Don Buse,IND,PG,43.326592,25,84,NaN,40.2,4.8,10.6,...,3.4,4.4,5.4,39.000000,0.977857,1.618526,2.176103,1.765386,-0.202005,6.335867
9,Billy Knight,IND,SF,42.781272,23,70,NaN,39.6,11.1,22.4,...,-1.0,2.8,3.4,39.000000,3.105842,1.011934,1.144775,1.171595,-0.202005,6.232141
10,George Gervin,SAS,SF,40.124420,23,81,NaN,33.9,8.7,17.5,...,0.0,2.6,3.2,50.000000,1.696969,1.275670,1.041642,1.097371,0.608660,5.720312
